### Import Libraries

In [1]:
import pandas as pd
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import janitor
from janitor import clean_names

### Import Data

In [3]:
# ------- define paths -------
# define relative path
relative_path = Path('../../data/raw_data/')

# get absolute path
absolute_path = relative_path.resolve()
print(absolute_path)

# declare file names
filename_iac = "IAC_Database_20250208.xls"
filename_ppi = "ARC_PPI_Draft.xlsx"
filename_generation = "annual_generation_state.xls"
filename_emissions = "emission_annual.xlsx"

# ------- import data -------

# import IAC database
iac_df = pd.read_excel(absolute_path/filename_iac, sheet_name=None)
# import all RECC* sheets from the IAC database excel file  
all_sheets = pd.read_excel(absolute_path/filename_iac, sheet_name=None) 
# filter sheets that match the pattern
recc_sheets = {name: data for name, data in all_sheets.items() if name.startswith('RECC')}
# combine matching sheets into a single DataFrame
iac_recc_df = pd.concat(
    [sheet.assign(RECC=name) for name, sheet in recc_sheets.items()],
    ignore_index=True
)

# import ASSESS table
iac_assess_df = all_sheets['ASSESS']

# import PPI sheet
ppi_df = pd.read_excel(absolute_path/filename_ppi, sheet_name="PPI")

# import Electricity Generation table
generation_df = pd.read_excel(absolute_path/filename_generation, sheet_name="Net_Generation_1990-2023 Final", skiprows=1)

# import Electricity Emissions table
emissions_df = pd.read_excel(absolute_path/filename_emissions, sheet_name="State Emissions")



/Users/yoselyn_ramirez/MEDS/Capstone/industrialenergy_datainterface/data/raw_data


### Normalize data

#### Transform the iac_recc table from wide to long format

Requirements
1. Keep all common columns
2. Create four rows for each input row (one for each energy source usage ranking: Primary, Secondary, Tertiary, Quaternary)
3. Maintain the relationship between energy source codes and their associated values: SOURCCODE, CONSERVED, SOURCONSV, SAVED
4. Order the columns to maintain the original dataframe structure

In [5]:
# Create a function to trasnform the recc table from wide to long format
def transform_recc_data(df):
    """
    Transform wide format usage data to long format by unpivoting usage-related columns.
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame in wide format
    
    Returns:
    pandas.DataFrame: Transformed DataFrame in long format
    """
    
    # Common columns that will be repeated for each usage record
    common_cols = ['SUPERID', 'ID', 'AR_NUMBER', 'APPCODE', 'ARC2', 
                  'IMPSTATUS', 'IMPCOST', 'REBATE', 'INCREMNTAL', 
                  'FY', 'IC_CAPITAL', 'IC_OTHER', 'PAYBACK', 'BPTOOL']
    
    # Create list of usage types
    usage_types = ['P', 'S', 'T', 'Q']
    
    # Initialize list to store transformed data
    transformed_data = []
    
    # Iterate through each row in the original dataframe
    for _, row in df.iterrows():
        # For each usage type, create a new record
        for usage in usage_types:
            new_row = {col: row[col] for col in common_cols}
            
            # Add usage-specific columns
            sourccode_col = f'{usage}SOURCCODE'
            conserved_col = f'{usage}CONSERVED'
            sourconsv_col = f'{usage}SOURCONSV'
            saved_col = f'{usage}SAVED'
            
            new_row['SOURCE_RANK'] = f'{usage}SOURCCODE'
            new_row['SOURCCODE'] = row.get(sourccode_col, '')
            new_row['CONSERVED'] = row.get(conserved_col, '')
            new_row['SOURCONSV'] = row.get(sourconsv_col, '')
            new_row['SAVED'] = row.get(saved_col, '')
            
            transformed_data.append(new_row)
    
    # Create new dataframe from transformed data
    result_df = pd.DataFrame(transformed_data)
    
    # Ensure columns are in the desired order
    column_order = common_cols[:7] + ['SOURCE_RANK', 'SOURCCODE', 'CONSERVED', 
                                    'SOURCONSV', 'SAVED'] + common_cols[7:]
    
    return result_df[column_order]

In [6]:
# Transform recc dataset from wide to long
iac_recc_tidy_df = transform_recc_data(iac_recc_df)

In [7]:
# Verify transformed data
filtered_df = iac_recc_tidy_df.query('SUPERID in ["AM000202","AM000504"]')
filtered_df

sample_recc_df = filtered_df[['SUPERID', 'ID', 'AR_NUMBER','IMPSTATUS', 'IMPCOST',
       'SOURCE_RANK', 'SOURCCODE', 'CONSERVED', 
                                    'SOURCONSV', 'SAVED']]
sample_recc_df

,SUPERID,ID,AR_NUMBER,IMPSTATUS,IMPCOST,SOURCE_RANK,SOURCCODE,CONSERVED,SOURCONSV,SAVED
32,AM000202,AM0002,2,I,60000.0,PSOURCCODE,E1,1077960.0,11049.0,52212.0
33,AM000202,AM0002,2,I,60000.0,SSOURCCODE,E2,10208.0,NaN,42872.0
34,AM000202,AM0002,2,I,60000.0,TSOURCCODE,NaN,NaN,NaN,NaN
35,AM000202,AM0002,2,I,60000.0,QSOURCCODE,NaN,NaN,NaN,NaN
152,AM000504,AM0005,4,N,960.0,PSOURCCODE,E2,83.0,NaN,273.0
153,AM000504,AM0005,4,N,960.0,SSOURCCODE,NaN,NaN,NaN,NaN
154,AM000504,AM0005,4,N,960.0,TSOURCCODE,R2,NaN,NaN,157.0
155,AM000504,AM0005,4,N,960.0,QSOURCCODE,NaN,NaN,NaN,NaN


#### Transform the iac_assess table from wide to long format

Requirements
1. Keep all common columns
2. Convert *_plant_usage and *_plant_cost columns into rows under the plant_usage and plant_cost columns, and add a separate column for the source code.
4. Order the columns to maintain the original dataframe structure

In [8]:
def transform_assess_data(df):
    """
    Transform wide format plant data to long format by converting *_plant_usage 
    and *_plant_cost columns into rows.
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame in wide format
    
    Returns:
    pandas.DataFrame: Transformed DataFrame in long format
    """
    # Common columns that will be preserved
    id_vars = ['CENTER', 'FY', 'SIC', 'NAICS', 'STATE', 'SALES', 
               'EMPLOYEES', 'PLANT_AREA', 'PRODUCTS', 'PRODUNITS', 
               'PRODLEVEL', 'PRODHOURS', 'NUMARS']
    
    # Melt cost columns
    cost_df = pd.melt(
        df,
        id_vars=['ID'] + id_vars,
        value_vars=[col for col in df.columns if col.endswith('_plant_cost')],
        var_name='source_code',
        value_name='plant_cost'
    )
    # Clean up source_code by removing '_plant_cost'
    cost_df['source_code'] = cost_df['source_code'].str.replace('_plant_cost', '')
    
    # Melt usage columns
    usage_df = pd.melt(
        df,
        id_vars=['ID'] + id_vars,
        value_vars=[col for col in df.columns if col.endswith('_plant_usage')],
        var_name='source_code',
        value_name='plant_usage'
    )
    # Clean up source_code by removing '_plant_usage'
    usage_df['source_code'] = usage_df['source_code'].str.replace('_plant_usage', '')
    
    # Merge cost and usage dataframes
    result_df = cost_df.merge(
        usage_df,
        on=['ID'] + id_vars + ['source_code'],
        how='outer'
    )
    
    # Create ordered categorical for source_code
    source_order = ['EC', 'ED', 'EF'] + [f'E{i}' for i in range(2, 13)] + [f'W{i}' for i in range(7)]
    result_df['source_code'] = pd.Categorical(result_df.source_code, categories=source_order, ordered=True)
    
    # Remove rows where both plant_cost and plant_usage are NA
    result_df = result_df.dropna(subset=['plant_cost', 'plant_usage'], how='all')

    # Sort by ID and source_code and set ID as index
    # result_df = result_df.sort_values(by=['ID', 'source_code']).set_index('ID')
    result_df = result_df.sort_values(by=['ID', 'source_code'])
    
    return result_df

In [9]:
# Transform assess dataset from wide to long
iac_assess_tidy_df = transform_assess_data(iac_assess_df)

In [10]:
# Verify transformed data
sample_assess_df = iac_assess_tidy_df.query('ID in ["AM0002","AM0005","AM0324"]')
sample_assess_df

,ID,CENTER,FY,SIC,NAICS,STATE,SALES,EMPLOYEES,PLANT_AREA,PRODUCTS,PRODUNITS,PRODLEVEL,PRODHOURS,NUMARS,source_code,plant_cost,plant_usage
32,AM0002,AM,1987,2761.0,NaN,TX,25000000.0,156.0,NaN,BUSINESS FORMS,NaN,NaN,2250.0,9,EC,267702.0,4867233.0
24,AM0002,AM,1987,2761.0,NaN,TX,25000000.0,156.0,NaN,BUSINESS FORMS,NaN,NaN,2250.0,9,E2,70657.0,19338.0
95,AM0005,AM,1987,2024.0,NaN,TX,12000000.0,39.0,NaN,ICE CREAM,5.0,2000.0,2080.0,7,EC,174617.0,2868113.0
87,AM0005,AM,1987,2024.0,NaN,TX,12000000.0,39.0,NaN,ICE CREAM,5.0,2000.0,2080.0,7,E2,10448.0,2903.0
6794,AM0324,AM,1998,3446.0,NaN,TX,24000000.0,140.0,99000.0,"Architectural metal work, mostly gratings and ...",NaN,NaN,5952.0,10,EC,141913.0,3808910.0
6795,AM0324,AM,1998,3446.0,NaN,TX,24000000.0,140.0,99000.0,"Architectural metal work, mostly gratings and ...",NaN,NaN,5952.0,10,ED,99270.0,16490.0
6786,AM0324,AM,1998,3446.0,NaN,TX,24000000.0,140.0,99000.0,"Architectural metal work, mostly gratings and ...",NaN,NaN,5952.0,10,E2,11950.0,2077.0
6800,AM0324,AM,1998,3446.0,NaN,TX,24000000.0,140.0,99000.0,"Architectural metal work, mostly gratings and ...",NaN,NaN,5952.0,10,W3,1840.0,1200.0
6801,AM0324,AM,1998,3446.0,NaN,TX,24000000.0,140.0,99000.0,"Architectural metal work, mostly gratings and ...",NaN,NaN,5952.0,10,W4,815045.0,5100000.0


#### Transform the ppi table from wide to long format

Requirements
1. Keep all common columns
2. Convert year columns into rows under the year and ppi columns
4. Order the columns to maintain the original dataframe structure

In [11]:
#ppi_tidy_df = transform_ppi_data(ppi_df)
ppi_tidy_df = pd.melt(
    ppi_df,
    id_vars=['ARC', 'Description', 'Series ID', 'Industry', 'Product'],
    value_vars=[1987, 1988, 1989, 1990, 1991, 1992,
                   1993, 1994, 1995, 1996, 1997, 1998,
                   1999, 2000, 2001, 2002, 2003, 2004,
                   2005, 2006, 2007, 2008, 2009, 2010,
                   2011, 2012, 2013, 2014, 2015, 2016,
                   2017, 2018],
    var_name='year',
    value_name='ppi'
    )

ppi_tidy_df = ppi_tidy_df.sort_values(by=['year', 'ARC'])

In [15]:
# Verify transformed data
ppi_tidy_df.head(-10)

,ARC,Description,Series ID,Industry,Product,year,ppi
0,2.1111,CONTROL PRESSURE ON STEAMER OPERATIONS,PCU334513334513,Industrial process variable instruments,Industrial process variable instruments,1987,114.4
1,2.1112,HEAT OIL TO PROPER TEMPERATURE FOR GOOD ATOMIZ...,NaN,NaN,NaN,1987,NaN
2,2.1113,REDUCE COMBUSTION AIR FLOW TO OPTIMUM,PCU334513334513,Industrial process variable instruments,Industrial process variable instruments,1987,114.4
3,2.1114,LIMIT AND CONTROL SECONDARY COMBUSTION AIR IN ...,PCU334513334513,Industrial process variable instruments,Industrial process variable instruments,1987,114.4
4,2.1115,ELIMINATE COMBUSTIBLE GAS IN FLUE GAS,PCU334513334513,Industrial process variable instruments,Industrial process variable instruments,1987,114.4
...,...,...,...,...,...,...,...
11185,2.8123,PAY UTILITY BILLS ON TIME,NaN,NaN,NaN,2018,NaN
11186,2.8211,CONSOLIDATE FREIGHT SHIPMENTS AND/OR DELIVERIES,NaN,NaN,NaN,2018,NaN
11187,2.8212,REDUCE DELIVERY SCHEDULES,NaN,NaN,NaN,2018,NaN
11188,2.8213,SIZE TRUCKS TO JOB,NaN,NaN,NaN,2018,NaN


### Transform the emissions table from wide to long format

Requirements
1. Keep all common columns
2. Convert emission type columns into rows under the emission type columns and emissions columns
3. Add a column for units
4. Order the columns to maintain the original dataframe structure


In [16]:
emissions_df.columns = [col.replace('\n(Metric Tons)', '') 
                        for col in emissions_df.columns]
# Melt the dataframe
emissions_tidy_df = pd.melt(
    emissions_df,
    id_vars = ['State', 'Year', 'Producer Type', 'Energy Source'],
    value_vars = ['CO2', 'SO2', 'NOx'],
    var_name = 'emission_type',
    value_name = 'amount'
    )

In [18]:
# Verify transformed data
emissions_tidy_df.head(-10)

,State,Year,Producer Type,Energy Source,emission_type,amount
0,AK,1990,Commercial Cogen,All Sources,CO2,824004
1,AK,1990,Commercial Cogen,Coal,CO2,821929
2,AK,1990,Commercial Cogen,Petroleum,CO2,2075
3,AK,1990,Commercial Non-Cogen,All Sources,CO2,0
4,AK,1990,Commercial Non-Cogen,Petroleum,CO2,0
...,...,...,...,...,...,...
147303,WY,2023,IPP NAICS-22 Non-Cogen,Natural Gas,NOx,0
147304,WY,2023,Industrial Non-Cogen,All Sources,NOx,373
147305,WY,2023,Industrial Non-Cogen,Natural Gas,NOx,373
147306,WY,2023,Industrial Cogen,All Sources,NOx,5843


### Clean Data

In [19]:
#------------------------ Clean data: ASSESS table ------------------------#

iac_assess_tidy_df = iac_assess_tidy_df.clean_names()
# strip whitespace from all string columns
for col in iac_assess_tidy_df.select_dtypes(include='object').columns:
    iac_assess_tidy_df[col] = iac_assess_tidy_df[col].str.strip()

#------------------------ Clean RECC table ------------------------#

# Replace old source coce for electricity values "E1" with "EC"
# Reason: E1 was replaced with EC, ED, and EF as of FY 95 (9/30/95)
# Reference: https://iac.university/technicalDocs/IAC_DatabaseManualv10.2.pdf
iac_recc_tidy_df.replace({'SOURCCODE':{'E1':'EC'}}, inplace=True)
iac_recc_tidy_df = iac_recc_tidy_df.clean_names()

#------------------------ Clean PPI table ------------------------#

ppi_tidy_df.rename(columns={'ARC': 'ARC2'}, inplace=True) # rename the column ARC to ARC2
ppi_tidy_df = ppi_tidy_df.clean_names()
# round ARC values to 4 decimal places
ppi_tidy_df['arc2'] = ppi_tidy_df['arc2'].round(4)
# replace "-" and "N/A" with 120 in the ppi column 
#--------- <TEMP until PPI values are collected> ----------#
ppi_tidy_df['ppi'] = ppi_tidy_df['ppi'].replace('-', 120)
ppi_tidy_df['ppi'].fillna(120, inplace=True)

#------------------------  Clean data: Electricity Generation table ------------------------#

generation_df = generation_df.rename(columns={'generation_megawatthours_': 'generation_megawatthours'})
generation_df['units'] = 'MWh' # add a column for units
generation_df = generation_df.clean_names()
# strip whitespace from all string columns
for col in generation_df.select_dtypes(include='object').columns:
    generation_df[col] = generation_df[col].str.strip()

#------------------------ Clean data: Electricity Emissions table ------------------------#

emissions_tidy_df = emissions_tidy_df.clean_names()
# strip whitespace from all string columns
for col in emissions_tidy_df.select_dtypes(include='object').columns:
   emissions_tidy_df[col] = emissions_tidy_df[col].str.strip()

/tmp/ipykernel_225542/1245211455.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ppi_tidy_df['ppi'] = ppi_tidy_df['ppi'].replace('-', 120)
/tmp/ipykernel_225542/1245211455.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ppi_tidy_df['ppi'].fillna(120, inplace=True)


### Save clean data

In [21]:
# IAC assess clean data
iac_assess_tidy_df.to_csv("../../data/intermediate_data/iac_assess_tidy.csv", index=False)
# IAC recc clean data
iac_recc_tidy_df.to_csv("../../data/intermediate_data/iac_recc_tidy.csv", index=False)
# PPI clean data
ppi_tidy_df.to_csv("../../data/intermediate_data/ppi_tidy.csv", index=False)
# Generation clean data
generation_df.to_csv("../../data/intermediate_data/generation.csv", index=False)
# Emissions clean data
emissions_tidy_df.to_csv("../../data/intermediate_data/emissions_tidy.csv", index=False)